In [27]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install grapheme
!pip install pandas
!pip install seaborn
!pip install scipy
!pip install --upgrade numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/76/ac/dea2939dfc3c591a2494121669455fd7d049248ef284c9542904ddbe05d5/numpy-1.26.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 12.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
papie 0.4.2 requires typing, which is not installed.
papie 0.4.2 requires numpy<1.24.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
pie-extended 0.1.2 requires numpy<1.24.0, but you have numpy 1.26.2 which is incompatible.


In [6]:
from glob import glob 
import os 
from itertools import combinations 
import grapheme 
from collatex import * 
from tqdm import tqdm 
import numpy as np 
import pandas as pd
import seaborn as sb 
import matplotlib.pyplot as plt 

from lxml import etree
from re import sub 
import xml.etree.ElementTree as ET
import string

In [7]:
sigles = [os.path.basename(fn).replace('xml_', '').replace('.xml', '') for fn in glob('../data/xml/*.xml')] 
sigles = sorted(sigles)
sigles

['A',
 'Ant',
 'B',
 'Br',
 'C',
 'D',
 'D2',
 'E',
 'F',
 'G',
 'Ge',
 'K',
 'L',
 'O',
 'W',
 'Y',
 'Z']

In [8]:
sigles = 'A','Ant','B', 'Br', 'C','D', 'D2','E', 'F', 'G', 'Ge', 'K', 'L', 'O', 'W', 'Y', 'Z'

In [9]:
def get_gap_lines(tree):
    gap_lines = []
    for text in tree.iterfind('.//' + "{" + NSMAP["MVN"] + "}" + 'text'):
        if 'n' in text.attrib:
            for line in text.iterfind('.//' + "{" + NSMAP["MVN"] + "}" + 'l'):
                if line.find('.//' + "{" + NSMAP["MVN"] + "}" + 'gap') is not None:
                    if 'n' in line.attrib:
                        n_value = line.attrib['n']
                        parts = n_value.split('_')
                        if len(parts) > 1:
                            k = "_".join(parts[1:])  # Join the parts after the first underscore
                        else:
                            k = n_value
                        gap_lines.append(k)

    return gap_lines

In [10]:
NSMAP = {'MVN': 'http://www.tei-c.org/ns/1.0'} 
removes = ('teiHeader', 'fw', 'supplied', 'abbr')
removes_expan_false = ('teiHeader', 'fw', 'supplied', 'ex', 'expan')
chars = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}       

In [11]:
import re
def extract_lines(xml_file, expan = True,  
                  punct = True, lower = True,
                  sep_abbr = True): 
    lines = {}
    tree = etree.parse(xml_file) 
    
    if expan:
        etree.strip_elements(tree, ("{"+ NSMAP["MVN"]+ "}" + s for s in removes), with_tail=False) 
    else: 
        etree.strip_elements(tree, ("{"+ NSMAP["MVN"]+ "}" + s for s in removes_expan_false), with_tail=False)
  
    context = etree.iterwalk(tree, events=("start", "end"))
    text = u"" 
    k = '' 
    
    for action, node in context: 
        tag_only = node.tag.replace("{http://www.tei-c.org/ns/1.0}","")  #remove ns for easier access
        if 'n' in node.attrib and tag_only == 'text': 
            title = node.attrib['n'] 

        if 'n' in node.attrib and tag_only == "l":
           # k = node.attrib['n']
            n_value = node.attrib['n']
            parts = n_value.split('_')
            if len(parts) > 1:
                k = "_".join(parts[1:])  
            else:
                k = n_value 
        # if a new pb (standalone element) is processed:
        if action == 'start' and tag_only == 'text': 
            continue
            
        elif action == 'start' and tag_only == 'lg':
            continue 
            
        # if new lb (standalone) is processed:
        elif action == 'start' and tag_only == 'lb':
            continue

        # list elements which you want to iterate through. this is not really neccessary.
        elif tag_only in ("group","text","MVN","body","cb","p","note"):
            continue

        # for all other elements, distinguish bet ween the start-event of the processing and
        # and the end-event. Attach the tail AFTER the child nodes were processed (= end-event) 
         
        elif action == 'start':
            #comment the following two lines out to not get the element markers
            #f.write(f"[{tag_only}]") 
            #text += f"[{tag_only}]"

            ############################################################################
            ########## filter out special characters, bars,                   ##########
            ########## superscript, or specific tags.                         ##########
            ############################################################################
                    
            
            #if a special glyph is present, encode it accordingly
                
            if tag_only == 'g':
                if sep_abbr:
                    if node.attrib['ref'] == '#bar': # ā, ē, ī, ō, ū, n̄ etc.
                        text += u'\u005f' #low line _

                    elif node.attrib['ref'] == '#apomod': # ʼ
                        text += u'\u02bc'

                    elif node.attrib['ref'] == '#usmod': # ꝰ
                        text += u'\ua770' 

                    elif node.attrib['ref'] == '#condes': # ꝯ
                        text += u'\ua76f'

                    elif node.attrib['ref'] == '#para': # ¶
                        text += u'\xb6'

                    elif node.attrib['ref'] == '#etfin': # ꝫ
                        text += u'\ua76b'

                    elif node.attrib['ref'] == '#pbardes': # ꝑ
                        text += '\ua751'

                    elif node.attrib['ref'] == '#pbardes': # ꝕ
                        text += u'\ua755'

                    elif node.attrib['ref'] == '#pflour': # ꝓ
                        text += u'\ua753'
                        
                    elif node.attrib['ref'] == '#rrot': #ꝛ
                        text += (u'\uA75B')
                    else:
                        text += str(node.attrib['ref']) # get the actual ref if there still are any left
                    
                else:
                    if node.attrib['ref'] == '#bar': # ā, ē, ī, ō, ū, n̄ etc.
                        text += u'\u0304'

                    elif node.attrib['ref'] == '#apomod': # ʼ
                        text += u'\u02bc'

                    elif node.attrib['ref'] == '#usmod': # ꝰ
                        text += u'\ua770'

                    elif node.attrib['ref'] == '#condes': # ꝯ
                        text += u'\ua76f'

                    elif node.attrib['ref'] == '#para': # ¶
                        text += u'\xb6'

                    elif node.attrib['ref'] == '#etfin': # ꝫ
                        text += u'\ua76b'

                    elif node.attrib['ref'] == '#pbardes': # ꝑ
                        text += u'\ua751'

                    elif node.attrib['ref'] == '#pbardes': # ꝕ
                        text += u'\ua755'

                    elif node.attrib['ref'] == '#pflour': # ꝓ
                        text += u'\ua753'
                        
                    elif node.attrib['ref'] == '#rrot': #ꝛ
                        text += (u'\uA75B')

                    else:
                        node.attrib['ref']
                        text += str(node.attrib['ref']) # get the actual ref if there still are any left

            # encode superscript letters
            superscript_dict = {'a':'ᵃ', 'b':'ᵇ', 'c':'ᶜ', 'd':'ᵈ', 'e':'ᵉ', 'f':'ᶠ',
                               'g':'ᵍ', 'h':'ʰ', 'i':'ᶦ', 'j':'ʲ', 'k':'ᵏ', 'l':'ˡ', 
                                'm':'ᵐ', 'n':'ⁿ', 'o':'ᵒ', 'p':'ᵖ', 'r':'ʳ', 's':'ˢ', 
                                't':'ᵗ', 'u':'ᵘ', 'v':'ᵛ', 'w':'ʷ', 'x':'ˣ', 'y': 'ʸ', 'z': 'ᶻ'}

            if tag_only == 'hi' and 'rend' in node.attrib and node.attrib['rend'] == 'superscript': #rend(ition) supplies information about the appearance of an element
                if node.text in superscript_dict:
                    text += str(superscript_dict[node.text]).strip()

            elif tag_only == 'ex':
                    text += str('*'+node.text+'€')

            # encode punctuation marks
            elif tag_only == 'pc':
                text += str(node.text).strip()

            # encode roman numerals
            elif tag_only == 'num':
                if node.text:
                    text += str('.'+node.text+'.').strip()

            # if there is still a node with text in it
            elif (node.text):
                text += node.text        

        # after the child elements
        elif action == 'end':
            #if there is a tail
            if (node.tail and node.tail not in "\t"): #if the tail is not yet in the text 
                #comment the following two lines out to not get the tail marker
                #text += "[tail]"
                #f.write("[tail]")
                #append to text-concatenation
                text += str(node.tail)

        if tag_only == 'l' or tag_only == 'lg':
            if k: 
                text = sub(r'\n', '', text) 
                
        if tag_only == 'lb':
            if k:
                text = sub(r'\n', '', text)
                if not punct:
                    punctuation_with_pilcrow = string.punctuation + '¶' + '⸫'
                    text = text.translate(str.maketrans('', '', punctuation_with_pilcrow))
                    text = text.translate(str.maketrans('', '', string.punctuation))
                if lower: 
                    text = text.lower()
                   
                    #text = text[::-1]
                  
                lines[k] = text 

                text = ''   
    # catch dangling last line (if applicable):
    if text:
        lines[k] = text
    text = sub(r'\n', '', text)  # Verwijder nieuwe regels

    if not punct:
        punctuation_with_pilcrow = string.punctuation + '¶' + '⸫'
        text = text.translate(str.maketrans('', '', punctuation_with_pilcrow))
        text = re.sub(r'\s+', ' ', text).strip()
        #text = text.translate(str.maketrans('', '', string.punctuation))  # Verwijder interpunctie
    if lower:
        text = text.lower()  # Zet om naar kleine letters
    lines[k] = text 
    num_orig_lines = len(lines)
    print(num_orig_lines)
    # remove lines with gaps:
    #gap_lines = get_gap_lines(tree)
    #lines = {k:v for k, v in lines.items() if k not in gap_lines}
    #print(f'-> removed {num_orig_lines - len(lines)} lines with gaps')
    #lines = {k:v for k, v in lines.items() if v.strip()} #if a line with a gap is removed, remove empty key, strip() removes spaces #The items() method returns a key-value pair
    
    return lines
    #num_orig_lines = len(lines)
    #print(num_orig_lines)
d = extract_lines(f'../data/xml/xml_{sigles[1]}.xml', expan = True, punct = True, lower = True)
print(d)

104
{'M1_47_606': '', 'M1_47_607': '', 'M1_47_608': 'ijt', 'M1_47_609': '', 'M1_47_610': '', 'M1_47_611': '', 'M1_48_612': '', 'M1_48_613': '', 'M1_48_614': '', 'M1_48_615': '', 'M1_48_616': 'jn', 'M1_48_617': '', 'M1_48_618': '', 'M1_48_619': '', 'M1_48_620': '', 'M1_48_621': '', 'M1_48_622': '', 'M1_48_623': '', 'M1_48_624': '', 'M1_49_625': '', 'M1_49_626': 't', 'M1_49_627': '', 'M1_49_628': '', 'M1_49_629': '', 'M1_49_630': '', 'M1_49_631': '', 'M1_49_632': '', 'M1_49_633': '', 'M1_49_634': '', 'M1_49_635': '', 'M1_49_636': '', 'M1_49_637': '', 'M1_50_638': '', 'M1_50_639': '', 'M1_50_640': '', 'M1_50_641': '', 'M1_50_642': '', 'M1_50_643': '', 'M1_50_644': 'soe viel ', 'M1_50_645': 'dus moch', 'M1_50_646': 'en*de€ clare', 'M1_50_647': 'miltheit h', 'M1_50_648': 'vrecheit l', 'M1_50_649': 'dit proeu', 'M1_50_650': 'die winn', 'M1_51_651': 'iacob e', 'M1_51_652': 'als d', 'M1_51_653': 'van gher', 'M1_51_654': 'doet d*er€ og', 'M1_51_655': 'oft comt', 'M1_51_656': 'dit euel t', 'M1_5

In [12]:
mss = {} 
for sigle in tqdm(sigles): 
    mss[sigle] = extract_lines(f'../data/xml/xml_{sigle}.xml',
                               expan = True, punct = True, lower = True,
                               sep_abbr = False) 
#print(mss)

 53%|█████████████████████████████████████████████████████████████████████▉                                                              | 9/17 [00:00<00:00, 42.82it/s]

1767
104
1816
604
1472
1811
247
276
1821


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 42.69it/s]

1770
148
67
700
1821
508
348
534


In [19]:
mss.keys()

dict_keys(['A', 'Ant', 'B', 'Br', 'C', 'D', 'D2', 'E', 'F', 'G', 'Ge', 'K', 'L', 'O', 'W', 'Y', 'Z'])

In [15]:
import re
idx = set()
for ms in mss:
    idx.update(set(mss[ms].keys()))
idx = sorted(idx)
witnesses = sorted(mss.keys())

lines = np.empty([len(idx), len(witnesses)], dtype="object")

for ms in mss.keys():
    for l in mss[ms]:
        lines[idx.index(l), witnesses.index(ms)] = mss[ms][l].replace('*', '<i>').replace('€', '</i>')

lines = pd.DataFrame(lines, index=idx, columns=witnesses)
lines.to_html('../data/xlsx/synoptic.html', escape=False) #import this html in Excel to get xlsx with exapansions in cursive 